In [1]:
%run _standard_imports.ipynb

python 3.4.3 |Anaconda 2.2.0 (64-bit)| (default, Mar  6 2015, 12:03:53) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.9.2
scipy 0.15.1
pandas 0.15.2
numexpr 2.3.1
pysam 0.8.3
pysamstats 0.23
petl 1.0.11
petlx 1.0.3
vcf 0.6.7
vcfnp 2.3.0.dev0
h5py 2.4.0
tables 3.1.1


In [2]:
# import stat
# # from sh import ssh
# # farm3 = ssh.bake("farm3-login")
# bsub = sh.Command('bsub')

In [2]:
ppq_bam_dir = '/nfs/team112_internal/production_files/Pf/ppq'
# pf_40_freeze_manifest_fn = '/nfs/team112_internal/production_files/Pf/4_0/pf_40_freeze_manifest.tab'
# samples_subset_fn = '/nfs/team112_internal/rp7/data/Pf/4_0/meta/PfCP_4.txt' # From Roberto March 30, 2106 14:14
# cinzia_metadata_fn = '/nfs/team112_internal/rp7/data/Pf/4_0/meta/PF_metadata_base.csv' # From Cinzia 22/03/2016 07:47
# cinzia_extra_metadata_fn = '/nfs/team112_internal/rp7/data/Pf/4_0/meta/PF_extrametadata.csv' # From Cinzia 22/03/2016 08:22
# compiled_metadata_3488_fn = '/nfs/team112_internal/rp7/data/Pf/4_0/meta/PfCP_4_3488_withtypes.txt'
output_dir = '/lustre/scratch109/malaria/rp7/data/ppq/breakpoint_spanning_reads'
# output_dir = '/nfs/team112_internal/rp7/data/Pf/4_0/pysamstats'
# !mkdir -p {"%s/scripts" % output_dir}
# !mkdir -p {"%s/results" % output_dir}



In [7]:
bam_files_dict = collections.OrderedDict()
import glob
for subdir in ['TRAC1', 'PFV2']:
    bam_files_dict[subdir] = glob.glob("%s/%s/*/*.bam" % (ppq_bam_dir, subdir))

In [9]:
bam_files_dict['TRAC1']

['/nfs/team112_internal/production_files/Pf/ppq/TRAC1/PH0502_C/pe.1.bam',
 '/nfs/team112_internal/production_files/Pf/ppq/TRAC1/PH0503_C/pe.1.bam',
 '/nfs/team112_internal/production_files/Pf/ppq/TRAC1/PH0504_C/pe.1.bam',
 '/nfs/team112_internal/production_files/Pf/ppq/TRAC1/PH0505_C/pe.1.bam',
 '/nfs/team112_internal/production_files/Pf/ppq/TRAC1/PH0506_C/pe.1.bam',
 '/nfs/team112_internal/production_files/Pf/ppq/TRAC1/PH0507_C/pe.1.bam',
 '/nfs/team112_internal/production_files/Pf/ppq/TRAC1/PH0508_C/pe.1.bam',
 '/nfs/team112_internal/production_files/Pf/ppq/TRAC1/PH0509_C/pe.1.bam',
 '/nfs/team112_internal/production_files/Pf/ppq/TRAC1/PH0510_C/pe.1.bam',
 '/nfs/team112_internal/production_files/Pf/ppq/TRAC1/PH0511_C/pe.1.bam',
 '/nfs/team112_internal/production_files/Pf/ppq/TRAC1/PH0512_C/pe.1.bam',
 '/nfs/team112_internal/production_files/Pf/ppq/TRAC1/PH0513_C/pe.1.bam',
 '/nfs/team112_internal/production_files/Pf/ppq/TRAC1/PH0514_C/pe.1.bam',
 '/nfs/team112_internal/production_fil

In [42]:
breakpoint_sequences = collections.OrderedDict()
breakpoint_sequences['0bp'] = 'GATAATCACAC'
breakpoint_sequences['1bp'] = 'CGATAATCACACT'
breakpoint_sequences['5bp'] = 'ATTACGATAATCACACTGTTG'
breakpoint_sequences['10bp'] = 'TTATGATTACGATAATCACACTGTTGGTTTC'
breakpoint_sequences['15bp'] = 'ACCGTTTATGATTACGATAATCACACTGTTGGTTTCGCCCT'
breakpoint_sequences['20bp'] = 'ATTTTACCGTTTATGATTACGATAATCACACTGTTGGTTTCGCCCTTGCCA'
breakpoint_sequences['5bp']

'ATTACGATAATCACACTGTTG'

In [13]:
from Bio.Seq import Seq
print(Seq(breakpoint_sequences['1bp']).reverse_complement())


AGTGTGATTATCG


In [43]:
import stat
bsub = sh.Command('bsub')

for breakpoint_sequence_name in breakpoint_sequences:
    breakpoint_reads_dir = '%s/plasmepsin_1_3/%s' % (output_dir, breakpoint_sequence_name)
    !mkdir -p {breakpoint_reads_dir}/results
    !mkdir -p {breakpoint_reads_dir}/scripts
    !mkdir -p {breakpoint_reads_dir}/logs

#     breakpoint_sequence = "'ATATAATTCAGGGTTTAGGGTTTA|TAAACCCTAAACCCTGAATTATAT'"
    breakpoint_sequence = "'%s|%s'" % (
        breakpoint_sequences[breakpoint_sequence_name],
        Seq(breakpoint_sequences[breakpoint_sequence_name]).reverse_complement()
    )
    for study in bam_files_dict:
        print("%s %s" % (study, breakpoint_sequence))
        for bam_fn in bam_files_dict[study]:
            ox_code = bam_fn.split('/')[7]
#             print("%s %s %s" % (study, ox_code, breakpoint_sequence))
            print('.', end='')
            num_breakpoint_reads_fn = "%s/results/num_breakpoint_reads_%s.txt" % (breakpoint_reads_dir, ox_code)
            if not os.path.exists(num_breakpoint_reads_fn):
                script_fn = "%s/scripts/nbpr_%s.sh" % (breakpoint_reads_dir, ox_code)
                fo = open(script_fn, 'w')
                print('samtools view %s | grep -E %s | wc -l > %s' % (bam_fn, breakpoint_sequence, num_breakpoint_reads_fn), file = fo)
                fo.close()
                st = os.stat(script_fn)
                os.chmod(script_fn, st.st_mode | stat.S_IEXEC)
                bsub(
                    '-G', 'malaria-dk',
                    '-P', 'malaria-dk',
                    '-q', 'normal',
                    '-o', '%s/logs/nbpr_%s.out' % (breakpoint_reads_dir, ox_code),
                    '-e', '%s/logs/nbpr_%s.err' % (breakpoint_reads_dir, ox_code),
                    '-J', 'nbpr_%s' % (ox_code),
                    '-R', "'select[mem>4000] rusage[mem=4000]'",
                    '-M', '4000',
                    script_fn
                )


TRAC1 'GATAATCACAC|GTGTGATTATC'
..................................................................................................................................................................................................................................................................................................PFV2 'GATAATCACAC|GTGTGATTATC'
...................................................................................................................................................................................................................................................................................................................TRAC1 'CGATAATCACACT|AGTGTGATTATCG'
..................................................................................................................................................................................................................................................................................................PFV2 'CGATAATC

In [46]:
num_breakpoints_dict = collections.OrderedDict()

for breakpoint_sequence_name in breakpoint_sequences:
    num_breakpoints_dict[breakpoint_sequence_name] = collections.OrderedDict()
    breakpoint_reads_dir = '%s/plasmepsin_1_3/%s' % (output_dir, breakpoint_sequence_name)

    for study in bam_files_dict:
        print("%s %s" % (study, breakpoint_sequence))
        for bam_fn in bam_files_dict[study]:
            ox_code = bam_fn.split('/')[7]
#             print("%s %s %s" % (study, ox_code, breakpoint_sequence))
            print('.', end='')
            num_breakpoint_reads_fn = "%s/results/num_breakpoint_reads_%s.txt" % (breakpoint_reads_dir, ox_code)
            fi = open(num_breakpoint_reads_fn, 'r')
            num_breakpoints_dict[breakpoint_sequence_name][ox_code] = int(fi.read())


TRAC1 'ATTTTACCGTTTATGATTACGATAATCACACTGTTGGTTTCGCCCTTGCCA|TGGCAAGGGCGAAACCAACAGTGTGATTATCGTAATCATAAACGGTAAAAT'
..................................................................................................................................................................................................................................................................................................PFV2 'ATTTTACCGTTTATGATTACGATAATCACACTGTTGGTTTCGCCCTTGCCA|TGGCAAGGGCGAAACCAACAGTGTGATTATCGTAATCATAAACGGTAAAAT'
...................................................................................................................................................................................................................................................................................................................TRAC1 'ATTTTACCGTTTATGATTACGATAATCACACTGTTGGTTTCGCCCTTGCCA|TGGCAAGGGCGAAACCAACAGTGTGATTATCGTAATCATAAACGGTAAAAT'
....................................................................

In [47]:
tbl_breakpoint_reads = (etl.wrap(zip(num_breakpoints_dict['0bp'].keys(), num_breakpoints_dict['0bp'].values())).pushheader(['ox_code', 'bp_reads_0bp'])
 .join(
        etl.wrap(zip(num_breakpoints_dict['1bp'].keys(), num_breakpoints_dict['1bp'].values())).pushheader(['ox_code', 'bp_reads_1bp']),
        key='ox_code')
 .join(
        etl.wrap(zip(num_breakpoints_dict['5bp'].keys(), num_breakpoints_dict['5bp'].values())).pushheader(['ox_code', 'bp_reads_5bp']),
        key='ox_code')
 .join(
        etl.wrap(zip(num_breakpoints_dict['10bp'].keys(), num_breakpoints_dict['10bp'].values())).pushheader(['ox_code', 'bp_reads_10bp']),
        key='ox_code')
 .join(
        etl.wrap(zip(num_breakpoints_dict['15bp'].keys(), num_breakpoints_dict['15bp'].values())).pushheader(['ox_code', 'bp_reads_15bp']),
        key='ox_code')
 .join(
        etl.wrap(zip(num_breakpoints_dict['20bp'].keys(), num_breakpoints_dict['20bp'].values())).pushheader(['ox_code', 'bp_reads_20bp']),
        key='ox_code')
)
tbl_breakpoint_reads.displayall()

ox_code,bp_reads_0bp,bp_reads_1bp,bp_reads_5bp,bp_reads_10bp,bp_reads_15bp,bp_reads_20bp
PH0157_C,1011,245,220,177,144,101
PH0161_C,706,103,90,67,54,33
PH0163_C,950,1,0,0,0,0
PH0167_C,937,1,0,0,0,0
PH0202_C,1030,0,0,0,0,0
PH0203_C,1485,1,0,0,0,0
PH0204_C,1047,2,0,0,0,0
PH0205_C,1026,0,0,0,0,0
PH0207_C,967,1,0,0,0,0
PH0209_C,1022,1,1,0,0,0


In [48]:
tbl_breakpoint_reads.select(lambda rec: rec[4] == 0 and rec[3] > 0)

ox_code,bp_reads_0bp,bp_reads_1bp,bp_reads_5bp,bp_reads_10bp,bp_reads_15bp,bp_reads_20bp
PH0209_C,1022,1,1,0,0,0
PH0590_C,364,2,1,0,0,0
PH0593_C,408,1,1,0,0,0
PH0597_C,364,2,1,0,0,0
PH0643_C,687,2,1,0,0,0


In [53]:
print(len(tbl_breakpoint_reads.data()))
print(len(tbl_breakpoint_reads.select(lambda rec: rec[4] > 0).data()))

597
169


In [55]:
169/597

0.2830820770519263

In [50]:
tbl_breakpoint_reads.totsv("%s/plasmepsin_1_3.tsv" % output_dir)

In [3]:
pf_5_0_manifest_fn = '/nfs/team112_internal/production/release_build/Pf/5_0_release_packages/pf_50_freeze_manifest_nolab_olivo.tab'

In [4]:
tbl_5_0_manifest = etl.fromtsv(pf_5_0_manifest_fn)
tbl_5_0_manifest

path,study,sample,bases,bases_mapped,avg_read_length,bases_of_1X_coverage,bases_of_5X_coverage,bases_of_10X_coverage,bases_of_50X_coverage,mean_coverage,sample_acc,run_acc
/lustre/scratch109/malaria/pfalciparum/output/5/e/3/7/247876/1_bam_merge/pe.1.bam,1094-PF-PDN-GH-AMENGA-ETEGO,PF0985-C,5285845400,4946434700,100,23117671,22948941,22787228,21141768,212.28,ERS444891,ERR586120
/lustre/scratch109/malaria/pfalciparum/output/a/4/e/5/108170/1_bam_merge/pe.1.bam,TRAC1,PD0509-Cx2,549490800,443334400,100,22419317,19526089,15598132,955025,19.46,ERS336350,ERR388789
/lustre/scratch109/malaria/pfalciparum/output/0/f/6/3/108533/1_bam_merge/pe.1.bam,1101-PF-CD-ONYAMBOKO,QG0031-C,2953693600,2711478400,100,22946958,22643437,22143320,16876098,117.18,ERS327030,ERR404158
/lustre/scratch109/malaria/pfalciparum/output/2/9/3/3/108631/1_bam_merge/pe.1.bam,1101-PF-CD-ONYAMBOKO,QG0143-C,2456156600,383948500,100,22523321,19576282,15999363,112353,16.64,ERS327159,ERR377536
/lustre/scratch109/malaria/pfalciparum/output/0/a/3/8/45325/1_bam_merge/pe.1.bam,PF20,PV0046-C,4375024512,3749292496,76,23094192,22940836,22838674,22321968,160.65,ERS013088,ERR022918


In [60]:
# for breakpoint_sequence_name in breakpoint_sequences:
for breakpoint_sequence_name in ['10bp']:
    breakpoint_reads_dir = '%s/plasmepsin_1_3_pf_5_0/%s' % (output_dir, breakpoint_sequence_name)
    !mkdir -p {breakpoint_reads_dir}/results
    !mkdir -p {breakpoint_reads_dir}/scripts
    !mkdir -p {breakpoint_reads_dir}/logs

    breakpoint_sequence = "'%s|%s'" % (
        breakpoint_sequences[breakpoint_sequence_name],
        Seq(breakpoint_sequences[breakpoint_sequence_name]).reverse_complement()
    )
    for rec in tbl_5_0_manifest:
        bam_fn = rec[0]
        ox_code = rec[2]
        print('.', end='')
        num_breakpoint_reads_fn = "%s/results/num_breakpoint_reads_%s.txt" % (breakpoint_reads_dir, ox_code)
        if not os.path.exists(num_breakpoint_reads_fn):
            script_fn = "%s/scripts/nbpr_%s.sh" % (breakpoint_reads_dir, ox_code)
            fo = open(script_fn, 'w')
            print('samtools view %s | grep -E %s | wc -l > %s' % (bam_fn, breakpoint_sequence, num_breakpoint_reads_fn), file = fo)
            fo.close()
            st = os.stat(script_fn)
            os.chmod(script_fn, st.st_mode | stat.S_IEXEC)
            bsub(
                '-G', 'malaria-dk',
                '-P', 'malaria-dk',
                '-q', 'normal',
                '-o', '%s/logs/nbpr_%s.out' % (breakpoint_reads_dir, ox_code),
                '-e', '%s/logs/nbpr_%s.err' % (breakpoint_reads_dir, ox_code),
                '-J', 'nbpr_%s' % (ox_code),
                '-R', "'select[mem>4000] rusage[mem=4000]'",
                '-M', '4000',
                script_fn
            )



........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [5]:
pf_5_0_breakpoint_reads = collections.OrderedDict()

# for breakpoint_sequence_name in breakpoint_sequences:
for breakpoint_sequence_name in ['10bp']:
    pf_5_0_breakpoint_reads[breakpoint_sequence_name] = collections.OrderedDict()
    breakpoint_reads_dir = '%s/plasmepsin_1_3_pf_5_0/%s' % (output_dir, breakpoint_sequence_name)
    for rec in tbl_5_0_manifest:
        bam_fn = rec[0]
        ox_code = rec[2]
        print('.', end='')
        num_breakpoint_reads_fn = "%s/results/num_breakpoint_reads_%s.txt" % (breakpoint_reads_dir, ox_code)
        fi = open(num_breakpoint_reads_fn, 'r')
        pf_5_0_breakpoint_reads[breakpoint_sequence_name][ox_code] = int(fi.read())



........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [7]:
pf_5_0_breakpoint_reads['10bp']

OrderedDict([('sample', 0), ('PF0985-C', 0), ('PD0509-Cx2', 0), ('QG0031-C', 0), ('QG0143-C', 0), ('PV0046-C', 0), ('QG0108-C', 0), ('QG0044-C', 0), ('QP0127-C', 0), ('PD1046-C', 0), ('QG0040-C', 0), ('PV0158-C', 0), ('PH0122-CW', 29), ('PT0223-Cx', 0), ('PE0109-C', 0), ('QP0115-C', 0), ('PD0071-C', 0), ('PZ0010-CW', 0), ('PH0499-C', 0), ('PD0667-C', 50), ('PF0211-C', 0), ('PE0215-C', 0), ('PD0009-01', 0), ('PM0144-C', 0), ('QP0244-C', 0), ('PD1054-C', 0), ('PF0886-C', 0), ('PD1097-C', 0), ('QP0158-C', 0), ('PV0291-C', 0), ('PF0366-C', 0), ('PJ0132-Cx', 0), ('PF0902-C', 0), ('PF0632-C', 0), ('QG0204-C', 0), ('PV0246-C', 0), ('PD0813-C', 0), ('PM0002-C', 0), ('PF0919-C', 0), ('PF0964-C', 0), ('PD1084-C', 0), ('PE0399-C', 0), ('PF0826-C', 0), ('PD0080-C', 0), ('PF1090-C', 0), ('PD0931-C', 0), ('PC0197-C', 0), ('PF0879-C', 0), ('PH0246-C', 0), ('PD0726-C', 0), ('PT0077-C', 0), ('QP0194-C', 0), ('PH1029-C', 0), ('PT0173-C', 0), ('PC0069-C', 0), ('PH0498-C', 0), ('PN0030-C', 0), ('PH0611-C'

In [34]:
tbl_5_0_breakpoint_reads = etl.wrap(
    zip(pf_5_0_breakpoint_reads['10bp'].keys(), pf_5_0_breakpoint_reads['10bp'].values())
).setheader(['ox_code', 'bp_reads_10bp'])
print(len(tbl_5_0_breakpoint_reads.data()))


6073


In [35]:
tbl_5_0_breakpoint_reads = etl.wrap(
    zip(pf_5_0_breakpoint_reads['10bp'].keys(), pf_5_0_breakpoint_reads['10bp'].values())
).setheader(['ox_code', 'bp_reads_10bp'])
print(len(tbl_5_0_breakpoint_reads.selectgt('bp_reads_10bp', 0).data()))


335


In [38]:
tbl_5_0_breakpoint_reads = etl.wrap(
    zip(pf_5_0_breakpoint_reads['10bp'].keys(), pf_5_0_breakpoint_reads['10bp'].values())
).setheader(['ox_code', 'bp_reads_10bp'])
print(len(tbl_5_0_breakpoint_reads.selectgt('bp_reads_10bp', 1).data()))


309


In [37]:
tbl_5_0_breakpoint_reads = etl.wrap(
    zip(pf_5_0_breakpoint_reads['10bp'].keys(), pf_5_0_breakpoint_reads['10bp'].values())
).setheader(['ox_code', 'bp_reads_10bp'])
tbl_5_0_breakpoint_reads

ox_code,bp_reads_10bp
QP0127-C,0
PD1046-C,0
QG0040-C,0
PV0158-C,0
PH0122-CW,29


In [36]:
tbl_5_0_breakpoint_reads = etl.wrap(
    zip(pf_5_0_breakpoint_reads['10bp'].keys(), pf_5_0_breakpoint_reads['10bp'].values())
).setheader(['ox_code', 'bp_reads_10bp'])
tbl_5_0_breakpoint_reads.totsv("%s/plasmepsin_1_3_pf_5_0.tsv" % output_dir)

In [29]:
tbl_5_0_breakpoint_reads

StopIteration: 

StopIteration: 

In [16]:
output_dir

'/lustre/scratch109/malaria/rp7/data/ppq/breakpoint_spanning_reads'

In [14]:
tbl_5_0_breakpoint_reads.selectgt('bp_reads_10bp', 0).displayall()

ox_code,bp_reads_10bp
PH0545-C,31
PH0683-C,23
PH0223-CW,1
PH0224-C,17
PH0578-C,22
PH0954-C,1
PH0225-C,21
PH0328-C,24
PH1155-C,18
PH0131-CW,23


In [ ]:
l /lustre/scratch109/malaria/rp7/data/Pf/4_0/breakpoint_spanning_reads/Pf3D7_13_v3_2830953/results/* | wc -l
bjobs | wc -l
!egrep -v '^0$' /lustre/scratch109/malaria/rp7/data/Pf/4_0/breakpoint_spanning_reads/Pf3D7_13_v3_2830953/results/*

In [37]:
print(Seq('TTGGTAGGTTTAGGGTTCAGGGTT').reverse_complement())

AACCCTGAACCCTAAACCTACCAA


In [57]:
# PV0097-C
print(Seq('TTGGTAGGTTTAGGGTTCAGGGTT').reverse_complement())

import stat
bsub = sh.Command('bsub')

breakpoint_reads_dir = '/lustre/scratch109/malaria/rp7/data/Pf/4_0/breakpoint_spanning_reads/Pf3D7_13_v3_2837144'
!mkdir -p {breakpoint_reads_dir}/results
!mkdir -p {breakpoint_reads_dir}/scripts
!mkdir -p {breakpoint_reads_dir}/logs

breakpoint_sequence = "'TTGGTAGGTTTAGGGTTCAGGGTT|AACCCTGAACCCTAAACCTACCAA'"
for rec in tbl_pf_40_freeze_manifest.data():
    bam_fn = rec[0]
    ox_code = rec[2].replace('-', '_')
    print('.', end='')
    num_breakpoint_reads_fn = "%s/results/num_breakpoint_reads_%s.txt" % (breakpoint_reads_dir, ox_code)
    if not os.path.exists(num_breakpoint_reads_fn):
        script_fn = "%s/scripts/nbpr_%s.sh" % (breakpoint_reads_dir, ox_code)
        fo = open(script_fn, 'w')
        print('samtools view %s | grep -E %s | wc -l > %s' % (bam_fn, breakpoint_sequence, num_breakpoint_reads_fn), file = fo)
        fo.close()
        st = os.stat(script_fn)
        os.chmod(script_fn, st.st_mode | stat.S_IEXEC)
        bsub(
            '-G', 'malaria-dk',
            '-P', 'malaria-dk',
            '-q', 'normal',
            '-o', '%s/logs/nbpr_%s.out' % (breakpoint_reads_dir, ox_code),
            '-e', '%s/logs/nbpr_%s.err' % (breakpoint_reads_dir, ox_code),
            '-J', 'nbpr_%s' % (ox_code),
            '-R', "'select[mem>4000] rusage[mem=4000]'",
            '-M', '4000',
            script_fn
        )


AACCCTGAACCCTAAACCTACCAA
...............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................